# Gradient Descent From Scratch

The optimization algorithm that powers all of deep learning.

**What you'll learn:**
- How gradient descent finds the minimum of a function
- What gradients are and how to compute them
- The effect of learning rate on convergence
- How to apply gradient descent to linear regression

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import animation
from IPython.display import HTML

plt.style.use('seaborn-v0_8-darkgrid')
%matplotlib inline

## 1. The Intuition: Rolling Downhill

Imagine you're blindfolded on a hilly landscape and want to find the lowest point:
- Feel the slope under your feet
- Take a step downhill
- Repeat until you stop going down

That's gradient descent!

In [ ]:
# Let's start with a simple 1D function: f(x) = x^2
# The minimum is obviously at x=0, but let's find it with gradient descent

def f(x):
    """Our simple function"""
    return x ** 2

def df(x):
    """The derivative (gradient) of f: df/dx = 2x"""
    return 2 * x

# Visualize
x_range = np.linspace(-5, 5, 100)
plt.figure(figsize=(10, 6))
plt.plot(x_range, f(x_range), 'b-', linewidth=2)
plt.xlabel('x')
plt.ylabel('f(x) = x²')
plt.title('Our Simple Function')
plt.axhline(y=0, color='k', linestyle='-', alpha=0.3)
plt.axvline(x=0, color='k', linestyle='-', alpha=0.3)
plt.show()

## 2. The Gradient Descent Algorithm

```
x_new = x_old - learning_rate * gradient
```

- **gradient**: The slope at current position (tells us which way is uphill)
- **learning_rate**: How big a step we take
- **minus sign**: We go *opposite* to the gradient (downhill)

In [ ]:
def gradient_descent_1d(f, df, x_start, learning_rate, n_steps):
    """
    Run gradient descent on a 1D function.
    
    Returns history of x values and function values.
    """
    x = x_start
    history = [(x, f(x))]
    
    for _ in range(n_steps):
        gradient = df(x)
        x = x - learning_rate * gradient
        history.append((x, f(x)))
    
    return history

# Run gradient descent starting from x=4
history = gradient_descent_1d(f, df, x_start=4, learning_rate=0.1, n_steps=20)

# Plot the path
plt.figure(figsize=(10, 6))
plt.plot(x_range, f(x_range), 'b-', linewidth=2, label='f(x) = x²')

# Plot the steps
xs, ys = zip(*history)
plt.plot(xs, ys, 'ro-', markersize=8, label='Gradient descent path')

plt.xlabel('x')
plt.ylabel('f(x)')
plt.title('Gradient Descent Finding the Minimum')
plt.legend()
plt.show()

print(f"Started at x={history[0][0]:.4f}, f(x)={history[0][1]:.4f}")
print(f"Ended at x={history[-1][0]:.4f}, f(x)={history[-1][1]:.4f}")

## 3. The Learning Rate Matters!

Let's see what happens with different learning rates.

In [ ]:
fig, axes = plt.subplots(1, 3, figsize=(15, 5))

learning_rates = [0.01, 0.1, 0.9]
titles = ['Too Small (0.01)', 'Just Right (0.1)', 'Too Large (0.9)']

for ax, lr, title in zip(axes, learning_rates, titles):
    history = gradient_descent_1d(f, df, x_start=4, learning_rate=lr, n_steps=20)
    xs, ys = zip(*history)
    
    ax.plot(x_range, f(x_range), 'b-', linewidth=2)
    ax.plot(xs, ys, 'ro-', markersize=6)
    ax.set_xlabel('x')
    ax.set_ylabel('f(x)')
    ax.set_title(f'{title}\nFinal x={xs[-1]:.4f}')
    ax.set_ylim(-1, 20)

plt.tight_layout()
plt.show()

## 4. Applying to Linear Regression

Now let's use gradient descent to fit a linear regression, instead of the closed-form solution.

For MSE loss with linear regression:
- ∂MSE/∂m = -(2/n) * Σ(y - ŷ) * x
- ∂MSE/∂b = -(2/n) * Σ(y - ŷ)

In [ ]:
# Generate the same data as the linear regression notebook
np.random.seed(42)
X = np.random.uniform(0, 10, 50)
true_slope, true_intercept = 2, 1
noise = np.random.normal(0, 2, 50)
y = true_slope * X + true_intercept + noise

plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.7)
plt.xlabel('X')
plt.ylabel('y')
plt.title('Our Data')
plt.show()

In [ ]:
def compute_gradients(X, y, m, b):
    """
    Compute gradients of MSE with respect to m and b.
    
    These are the partial derivatives of the loss function.
    """
    n = len(X)
    y_pred = m * X + b
    error = y - y_pred
    
    # Gradients
    dm = -(2/n) * np.sum(error * X)
    db = -(2/n) * np.sum(error)
    
    return dm, db

def mse(y_true, y_pred):
    return np.mean((y_true - y_pred) ** 2)

def gradient_descent_linear(X, y, learning_rate=0.01, n_steps=1000):
    """
    Fit linear regression using gradient descent.
    """
    # Initialize with random values
    m, b = 0.0, 0.0
    
    history = []
    
    for step in range(n_steps):
        # Calculate gradients
        dm, db = compute_gradients(X, y, m, b)
        
        # Update parameters
        m = m - learning_rate * dm
        b = b - learning_rate * db
        
        # Track progress
        y_pred = m * X + b
        loss = mse(y, y_pred)
        history.append({'step': step, 'm': m, 'b': b, 'loss': loss})
    
    return m, b, history

In [ ]:
# Run gradient descent
m_gd, b_gd, history = gradient_descent_linear(X, y, learning_rate=0.01, n_steps=1000)

print(f"Gradient Descent: m={m_gd:.4f}, b={b_gd:.4f}")
print(f"True values:      m={true_slope}, b={true_intercept}")

In [ ]:
# Plot the loss over time
losses = [h['loss'] for h in history]

plt.figure(figsize=(10, 6))
plt.plot(losses)
plt.xlabel('Step')
plt.ylabel('MSE Loss')
plt.title('Loss Decreasing Over Training')
plt.yscale('log')  # Log scale to see the detail
plt.show()

print(f"Initial loss: {losses[0]:.4f}")
print(f"Final loss: {losses[-1]:.4f}")

In [ ]:
# Visualize the parameter trajectory
ms = [h['m'] for h in history]
bs = [h['b'] for h in history]

# Create loss surface
m_range = np.linspace(-1, 4, 100)
b_range = np.linspace(-3, 5, 100)
M, B = np.meshgrid(m_range, b_range)

MSE_grid = np.zeros_like(M)
for i in range(M.shape[0]):
    for j in range(M.shape[1]):
        y_pred = M[i,j] * X + B[i,j]
        MSE_grid[i,j] = mse(y, y_pred)

plt.figure(figsize=(10, 8))
contour = plt.contour(M, B, MSE_grid, levels=30)
plt.plot(ms, bs, 'r.-', markersize=2, linewidth=1, label='GD path')
plt.plot(ms[0], bs[0], 'go', markersize=10, label='Start')
plt.plot(ms[-1], bs[-1], 'r*', markersize=15, label='End')
plt.plot(true_slope, true_intercept, 'b^', markersize=12, label='True')
plt.xlabel('Slope (m)')
plt.ylabel('Intercept (b)')
plt.title('Gradient Descent Path on Loss Surface')
plt.legend()
plt.colorbar(contour, label='MSE')
plt.show()

In [ ]:
# Final visualization: our fitted line
plt.figure(figsize=(10, 6))
plt.scatter(X, y, alpha=0.7, label='Data')
plt.plot([0, 10], [b_gd, 10*m_gd + b_gd], 'r-', linewidth=2, 
         label=f'GD: y = {m_gd:.2f}x + {b_gd:.2f}')
plt.plot([0, 10], [true_intercept, 10*true_slope + true_intercept], 'g--', 
         linewidth=2, alpha=0.5, label=f'True: y = {true_slope}x + {true_intercept}')
plt.xlabel('X')
plt.ylabel('y')
plt.title('Linear Regression via Gradient Descent')
plt.legend()
plt.show()

## 5. Experiment: Learning Rate Effects

Try changing the learning rate and see what happens!

In [ ]:
fig, axes = plt.subplots(2, 2, figsize=(12, 10))

learning_rates = [0.001, 0.01, 0.05, 0.1]

for ax, lr in zip(axes.flat, learning_rates):
    _, _, hist = gradient_descent_linear(X, y, learning_rate=lr, n_steps=500)
    losses = [h['loss'] for h in hist]
    
    ax.plot(losses)
    ax.set_xlabel('Step')
    ax.set_ylabel('MSE')
    ax.set_title(f'Learning Rate = {lr}\nFinal Loss = {losses[-1]:.4f}')
    ax.set_yscale('log')

plt.tight_layout()
plt.show()

## Key Takeaways

1. **Gradient descent iteratively finds the minimum** by taking steps opposite to the gradient
2. **The learning rate is crucial** - too small is slow, too large diverges
3. **The gradient tells us the direction of steepest increase**, so we go the opposite way
4. **This is how neural networks learn** - same principle, more parameters

## What's Next?

- **Stochastic Gradient Descent (SGD)**: Use random subsets for efficiency
- **Momentum**: Build up speed in consistent directions
- **Adam**: Adaptive learning rates per parameter
- **Apply to neural networks**: Same idea, thousands of parameters

In [ ]:
# Summary
print("=" * 50)
print("GRADIENT DESCENT SUMMARY")
print("=" * 50)
print(f"Learning rate: 0.01")
print(f"Steps: 1000")
print(f"Final parameters: m={m_gd:.4f}, b={b_gd:.4f}")
print(f"True parameters: m={true_slope}, b={true_intercept}")
print(f"Final MSE: {losses[-1]:.4f}")
print("=" * 50)